In [12]:
import os
import pandas as pd
os.chdir(os.getcwd())

ModuleNotFoundError: No module named 'pandas'

In [9]:
dfs = {}
# 1. ファイル名を指定: 産業別、売上高経常利益率別常時従業者数
excel_dir = os.path.join(os.path.dirname(os.getcwd()), "downloads")
filepaths = [fp for fp in os.listdir(excel_dir) if "産業別、売上高経常利益率別常時従業者数" in fp]
for file_path in filepaths:
    year = file_path[-8:-4]
    full_path = os.path.join(os.path.dirname(os.getcwd()), f"downloads/{file_path}")
    # 2. Open the workbook and list sheets
    xls = pd.ExcelFile(full_path, engine='xlrd')
    print("Available sheets:", xls.sheet_names)

    # 3. Read and clean each sheet into a DataFrame

    for sheet in xls.sheet_names:
        # Adjust `header` and `skiprows` to match where your table's real header lives
        df = pd.read_excel(
            full_path,
            sheet_name=sheet,
            engine='xlrd',
            header=[0, 1],   # e.g. two header rows; change to [0] or None as needed
            skiprows=0       # e.g. if there are extra intro rows, bump this up
        )
        # Drop fully empty rows/columns
        df.dropna(how='all', inplace=True)
        df.dropna(axis=1, how='all', inplace=True)

        # If pandas created a MultiIndex for columns, flatten it
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [
                "_".join([str(c).strip() for c in col if str(c).strip()])
                for col in df.columns.values
            ]
        else:
            df.columns = [str(col).strip() for col in df.columns]

        if year == "2004" or year == "2005":
            # Merge rows 2, 3, and 4 into a single string for each column
            merged_headers = df.iloc[1:5].fillna('').astype(str).agg(' '.join, axis=0).str.strip()
            df = df.iloc[5:] 
            # set headers
            df.columns = merged_headers
            df.columns.values[0] = "年度"

            # Insert a new column before column 0
            df.insert(0, "産業", None)

            # Move values from the original column to the new column if "年度" is not in the original column
            df.loc[~df.iloc[:, 1].str.contains("年度", na=False), "産業"] = df.iloc[:, 1]

            # Clear the original column where values were moved
            df.loc[~df.iloc[:, 1].str.contains("年度", na=False), df.columns[1]] = None

            # Fill NaN values in column 1 with the value from the first row until another non-NaN value appears in column 0
            df.iloc[:, 0] = df.iloc[:, 0].ffill()
            
            # Drop rows where column 1 is NaN
            df = df.dropna(subset=[df.columns[1]])
        elif year == "2007":
            # Merge rows 2, 3, and 4 into a single string for each column
            merged_headers = df.iloc[1:4].fillna('').astype(str).agg(' '.join, axis=0).str.strip()
            df = df.iloc[4:] 
            df.columns = merged_headers
            df.columns.values[0] = "産業"
            df.columns.values[1] = "年度"
            # Fill NaN values in column 1 with the value from the first row until another non-NaN value appears in column 0
            df.iloc[:, 0] = df.iloc[:, 0].ffill()
            df = df.drop(df.columns[2], axis=1)
        elif year == "2009" or year == "2011" or year == "2012" or year == "2013":
            # Merge rows 2, 3, and 4 into a single string for each column
            merged_headers = df.iloc[0:3].fillna('').astype(str).agg(' '.join, axis=0).str.strip()
            df = df.iloc[3:] 
            df.columns = merged_headers
            df.columns.values[0] = "産業"
            df.columns.values[1] = "年度"
            # Fill NaN values in column 1 with the value from the first row until another non-NaN value appears in column 0
            df.iloc[:, 0] = df.iloc[:, 0].ffill()
        elif int(year) >= 2020:
            merged_headers = df.iloc[0:1].fillna('').astype(str).agg(' '.join, axis=0).str.strip()
            df = df.iloc[3:] 
            df.columns = merged_headers
            # drop column 0, 2
            df.columns.values[1] = "産業"
            df.columns.values[3] = "年度"
            df = df.drop(df.columns[0], axis=1)
        else: 
            # Merge rows 2, 3, and 4 into a single string for each column
            merged_headers = df.iloc[2:5].fillna('').astype(str).agg(' '.join, axis=0).str.strip()
            df = df.iloc[5:] 
            # set headers
            df.columns = merged_headers
            
            df.columns.values[0] = "産業"
            df.columns.values[1] = "年度"
            # Fill NaN values in column 1 with the value from the first row until another non-NaN value appears in column 0
            df.iloc[:, 0] = df.iloc[:, 0].ffill()
            
            # Drop rows where column 1 is NaN
            df = df.dropna(subset=[df.columns[1]])
            # drop column 2
            if year == "2003" or year == "2006" or year == "2008": 
                df = df.drop(df.columns[2], axis=1)

        # strip column 1 if possible
        try:    
            df.iloc[:, 1] = df.iloc[:, 1].str.strip()
        except AttributeError:
            pass
        
        dfs[f"{year}"] = df


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kohsukesagara/Desktop/graspp-25S-innovation/downloads'

In [ ]:
print(dfs.keys())
dfs["2020"]

dict_keys(['2019', '2018', '2020', '2008', '2009', '2021', '2004', '2010', '2011', '2005', '2013', '2007', '2006', '2012', '2016', '2003', '2017', '2015', '2014'])


,産業,年度,企業数,経常利益率<-10%,－10%≦経常利益率<-8%,－8%≦経常利益率<-6%,－6%≦経常利益率<-4%,－4%≦経常利益率<-2%,－2%≦経常利益率<0%,0%≦経常利益率≦2%,2%<経常利益率≦4%,4%<経常利益率≦6%,6%<経常利益率≦8%,8%<経常利益率≦10%,10%<経常利益率,合計
3,総合計,2015年度,30231,76216,41664,64067,81521,193552,612730,3804993,3314852,2644163,1432669,940394,2014781,15221602
4,総合計,2016年度,30151,83650,27321,44985,60609,165550,424227,3754856,3454044,2668775,1469932,897393,2088641,15139983
5,総合計,2017年度,29530,76549,18998,35020,75641,155220,452852,3622057,3128649,2676197,1761478,787581,2362377,15152619
6,総合計,2018年度,29780,85674,17469,50094,107515,185799,505794,3801199,3120867,2660768,1560029,893542,2413369,15402119
7,総合計,2019年度,29295,116289,38058,56487,121787,323376,817801,3606890,3123952,2480078,1332252,929381,2138668,15085019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,その他の産業,2015年度,980,1111,84,138,2439,4629,7372,80506,80063,82311,36851,24656,74557,394717
175,その他の産業,2016年度,955,1268,1211,1130,202,3874,4221,62678,83353,90945,42291,26421,83737,401331
176,その他の産業,2017年度,915,333,1425,51,346,1876,5031,80016,73083,59471,37639,27953,89167,376391
177,その他の産業,2018年度,902,5011,745,444,4533,3440,6976,75156,47262,69409,64419,25432,78061,380888


In [ ]:
# save dfs to csv
os.makedirs("../data/産業別、売上高経常利益率別常時従業者数", exist_ok=True)
for key, df in dfs.items():
    df.to_csv(f"../data/産業別、売上高経常利益率別常時従業者数/{key}.csv", index=True)